[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/decisionbranches/decisionbranches/blob/master/examples/pipeline.ipynb)


# Finding Needles in Massive Haystacks: Fast Search-By-Classification in Large-Scale Data Catalogs - Demo

<img src="https://github.com/decisionbranches/decisionbranches/raw/main/figures/pipeline.png" alt="Drawing" style="width: 400px;"/> \
This notebook guides through our whole search pipeline as shown in the figure. We use the **Satimage dataset** for illustrating how to use the pipeline. Therefore, no feature extraction of the data is required in this case.

## Download and install our model

In [ ]:
!pip install git+https://github.com/decisionbranches/decisionbranches.git



In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from decisionbranches.utils.helpers import generate_fidxs
from decisionbranches.models.boxSearch.boxClassifier import BoxClassifier
from py_kdtree.treeset import KDTreeSet

In [ ]:
seed=42
np.random.seed(seed)


#Parameter
nfeat = 10 #Size of feature subsets 
nind = 100 #Number of feature subsets
dbranch_cfg = {"top_down":False,"max_evals":"all","stop_infinite":True} 
#top_down: if top-down refinement phase should be enabled; max_evals: number of feature subsets to evaluate per iteration; stop_infinite: stop expansion until infinity

label = "4."

## Phase 1: Offline Preprocessing
In this preprocessing phase, we prepare the indexes required for our classifier. Note that this phase may take significantly longer for larger datasets but it only needs to be executed **once** to make our classifier operable. Afterwards, multiple queries can be executed without any additional preprocessing.

### Load features
We load the Satimage dataset from OpenML (https://www.openml.org/) and transform the dataset into a binary classification problem to make it useable for our model.

In [ ]:
X, y = fetch_openml('satimage', version=1, return_X_y=True, as_frame=False) #Load dataset from OpenML database

y_bin = np.zeros(len(y),dtype=int)
y_bin[y==label] = 1 #Make binary classification problem out of the dataset

X_train,X_test,y_train,y_test = train_test_split(X,y_bin,train_size=0.05,random_state=seed) #Split data into train and test set
print("Number of rare training objects: ",np.sum(y_train))
print("Number of points to query: ",len(X_test))

### Generate feature subsets (indexes)
We randomly generate the required feature subsets in this case but they can be also indiviually specified if required.

In [ ]:
subsets = generate_fidxs(n_feat=nfeat,n_ind=nind,feats=np.arange(X.shape[1]),seed=seed) 


### Build indexes
Multiple indexes are built based on the generated feature subsets from the step before. The *treeset* object abstracts all index structures for the user at once. The individual indexes (kd-trees) are stored under *path*. Under *path* each kd-tree is identified via the ids of the corresponding feature subset. For each kd-tree the tree structure (.pkl file) and their leaves (.mmap file) are stored.  

In [ ]:
treeset = KDTreeSet(subsets,path="./indexes/",leaf_size=60,verbose=False)
treeset.fit(X_test)

---

## Phase 2: Query Processing
In this phase, we simulate an examplary user query to find rare objects using our index-aware classifier. The user query consists of labeled rare (y=1) and non-rare (y=0) instances. In this case the labeled instances of the user query are contained in *X_train* (features) and *y_train* (labels). The data catalog on which the search is executed is represented by *X_test* and *y_test*. Note that in practice the data catalog should be many times larger to take full advantage of our search-by-classification approach.

### Train Index-aware Classifier
We provide our classifier (BoxClassifier) the indexes in which to perform the search via the parameter *indices*.


In [ ]:
dbranch = BoxClassifier(tot_feat=X.shape[1],indices=subsets,cfg=dbranch_cfg)

dbranch.fit(X_train,y_train) #Construct the boxes

preds = dbranch.predict(X_test)
print("Test F1-score: ",f1_score(y_test, preds)) # Search quality of our found boxes in the unknown data catalog

### Extract range queries (boxes)
We extract the found boxes. These are defined by their minimum point, maximum point and corresponding feature subset.

In [ ]:
mins,maxs,fidxs = dbranch.get_boxes()

### Query boxes
We query for the found boxes in the index structures. The function *multi_query_ranked_cy* performs the search for all boxes at once and sorts the found point based on their number of occurences (points that are contained in multiple boxes are thereby counted).

In [ ]:
inds,counts,time,loaded_leaves = treeset.multi_query_ranked_cy(mins,maxs,fidxs)

print("Number of found points: ",len(inds))
print("Loading time: ",time)
print("Number of loaded leaves: ",loaded_leaves)

---

## Ensemble
The same query is repeated for our ensemble classifier which consists of 25 classifiers. We can observe that the search quality increases at the cost of longer query time.

In [ ]:
from decisionbranches.models.boxSearch.ensemble import Ensemble
ens = Ensemble(tot_feat=X.shape[1],indices=subsets,n_estimators=25,cfg={"max_evals":"auto"}) #n_estimators: number of models in the ensemble

ens.fit(X_train,y_train)

preds = ens.predict(X_test)
print("Test F1-score: ",f1_score(y_test, preds))

### Extract range queries (boxes)

In [ ]:
mins,maxs,fidxs = ens.get_boxes()

### Query boxes

In [ ]:
inds,counts,time,loaded_leaves = treeset.multi_query_ranked_cy(mins,maxs,fidxs)

print("Number of found points: ",len(inds))
print("Loading time: ",time)
print("Number of loaded leaves: ",loaded_leaves)